# GP Cartpole Server

Genetic program for the cartpole REST interface. The GP creates a server program to reply to requests on the cartpole REST interface.

## Example: `cart_get()` Function

As a example, look at the `cart_get()` prototype the OpenAPI generator created for the cartpole REST interface.  

```python
def cart_get():  # noqa: E501
    """cart_get

     # noqa: E501


    :rtype: Cart
    """
    return 'do some magic!'
```

A concrete example implementation would look like the following:

```python
def cart_get():  # noqa: E501
    # just return a Cart object with random values

    position = random()
    velocity = random()
    direction = choice(["left", "right"]) 

    return Cart(position=position, velocity=velocity, direction=direction)
```

The `Cart` class is one the generated models. The contructor defines the params as follows:
```python
def __init__(self, position=None, velocity=None, direction=None):  # noqa: E501
    """Cart - a model defined in OpenAPI

    :param position: The position of this Cart.  # noqa: E501
    :type position: float
    :param velocity: The velocity of this Cart.  # noqa: E501
    :type velocity: float
    :param direction: The direction of this Cart.  # noqa: E501
    :type direction: str
    """
    ...
```

## Genetic Program Design

The GP bases on the Artificial Ant Problem from the [DEAP example site](https://deap.readthedocs.io/en/master/examples/gp_ant.html).

The idea is, if there already is a cart object and return this, otherwise create a new one and return this. Here's the pseudo code:

```
if cart exist 
    return cart
else
    pos = get_init_cart_position
    vel = get_init_cart_velocity
    dir = get_init_cart_direction
    return new Cart with params above
```

To evaluate the code, we test run it. It needs to run without any exception and comply with the expectations if a clients requests the interface. 

## Genetic Program Implementation

### Import libs

In [1]:
import random

import numpy

from functools import partial

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

### Functions

These are generic functions, e.g. for control flow and concatenating functions with each other, and the specific function for the server code. 

#### Generic Functions

In [2]:
def progn(*args):
    for arg in args:
        arg()

def prog2(out1, out2): 
    return partial(progn,out1,out2)

def prog3(out1, out2, out3):     
    return partial(progn,out1,out2,out3)

def if_then_else(condition, out1, out2):
    out1() if condition() else out2()

#### Server Code Function

In [7]:
class CartpoleServer(object):

    def __init__(self):
        self._cart = None

    # the init functions return random numbers
    def get_init_cart_position(self):
        return random()

    def get_init_cart_velocity(self):
        return random()

    # we know the values from the models, e.g.
    #   models/cart.py or
    #   models/direction.py
    def get_init_cart_direction(self):
        return choice(["left", "right"])

    def create_init_cart():
        position = self.get_init_cart_position()
        velocity = self.get_init_cart_velocity()
        direction = selfget_init_cart_direction()

        return Cart(position=position, velocity=velocity, direction=direction)

    def if_cart_exists():
        if (self._cart != None):
            return create_init_cart()
        else:
            return self._cart



#### Setup GP

In [ ]:
pset = gp.PrimitiveSet("MAIN", 0)
pset.addPrimitive(ant.if_food_ahead, 2)

pset.addPrimitive(prog2, 2)
pset.addPrimitive(prog3, 3)

pset.addTerminal(get_init_cart_position, 0)
pset.addTerminal(get_init_cart_velocity, 0)
pset.addTerminal(get_init_cart_direction, 0)

pset.addTerminal(ant.turn_left)
pset.addTerminal(ant.turn_right)